# Proyecto Equipo 5
# Uso del Módulo 4 - Machine Learning
## Prueba SVM con kernel RBF

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
import warnings
warnings.filterwarnings("ignore")

## Lectura y organización de los datos

In [2]:
df = pd.read_csv('../archive/data_features.csv')
df

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_HOUSING_TYPE,MONTHS_EMPLOYED,timespan,paid_off,pastdue_1-29,pastdue_30-59,pastdue_60-89,pastdue_90-119,pastdue_120-149,pastdue_over_150,no_loan,Score,label,bin_labels
0,5008804,1,1,427500.0,Rented apartment,151,16,13,1,1,0,0,0,0,1,783.541667,4,1
1,5008805,1,1,427500.0,Rented apartment,151,15,12,1,1,0,0,0,0,1,774.375000,4,1
2,5008806,1,1,112500.0,House / apartment,37,30,7,7,0,0,0,0,0,16,733.125000,3,0
3,5008808,0,1,270000.0,House / apartment,101,5,0,2,0,0,0,0,0,3,612.583333,2,0
4,5008809,0,1,270000.0,House / apartment,101,5,0,0,0,0,0,0,0,5,612.583333,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5150482,1,1,157500.0,House / apartment,57,18,6,12,0,0,0,0,0,0,850.000000,5,1
36453,5150483,1,1,157500.0,House / apartment,57,18,0,0,0,0,0,0,0,18,753.750000,4,1
36454,5150484,1,1,157500.0,House / apartment,57,13,1,12,0,0,0,0,0,0,804.166667,5,1
36455,5150485,1,1,157500.0,House / apartment,57,2,0,2,0,0,0,0,0,0,703.333333,3,0


In [3]:
#Asignación de características y etiquetas a X y Y respectivamente
X = df[['FLAG_OWN_CAR','FLAG_OWN_REALTY','AMT_INCOME_TOTAL','MONTHS_EMPLOYED','timespan']].values
y = df['bin_labels'].values
print(f'Tamaño X: {X.shape}\nTamaño y: {y.shape}')

Tamaño X: (36457, 5)
Tamaño y: (36457,)


## Selección de sets de entrenamiento y prueba por K-fold, 5 iteraciones

In [4]:
#Arreglos para guardar los datos de K-fold
list_X_train = []
list_X_test = []
list_y_train = []
list_y_test = []
#División de índices por k-fold
num_pliegues = 5
kf = KFold(n_splits=num_pliegues, shuffle=True, random_state=42)
for i, (train_indices, test_indices) in enumerate(kf.split(X)):
    X_entrenamiento, X_prueba = X[train_indices], X[test_indices]
    y_entrenamiento, y_prueba = y[train_indices], y[test_indices]

    # Imprimir información sobre el pliegue actual
    print(f"Pliegue {i + 1}:")
    print(f"Tamaño del conjunto de entrenamiento: {len(X_entrenamiento)}")
    print(f"Tamaño del conjunto de prueba: {len(X_prueba)}")
    print("\n")
    list_X_train.append(X_entrenamiento)
    list_X_test.append(X_prueba)
    list_y_train.append(y_entrenamiento)
    list_y_test.append(y_prueba)

Pliegue 1:
Tamaño del conjunto de entrenamiento: 29165
Tamaño del conjunto de prueba: 7292


Pliegue 2:
Tamaño del conjunto de entrenamiento: 29165
Tamaño del conjunto de prueba: 7292


Pliegue 3:
Tamaño del conjunto de entrenamiento: 29166
Tamaño del conjunto de prueba: 7291


Pliegue 4:
Tamaño del conjunto de entrenamiento: 29166
Tamaño del conjunto de prueba: 7291


Pliegue 5:
Tamaño del conjunto de entrenamiento: 29166
Tamaño del conjunto de prueba: 7291




## Funciones para entrenar modelos y obtener métricas

In [5]:
#Función de entrenamiento y prueba para el modelo
def train_test_modelo(modelo, X_train, y_train, X_test, y_test):
	modelo.fit(X_train,y_train)
	pred_train = modelo.predict(X_train)
	confmat_train = confusion_matrix(y_train, pred_train)
	pred_test = modelo.predict(X_test)
	confmat_test = confusion_matrix(y_test, pred_test)
	return confmat_train, confmat_test


In [6]:
def obtener_metricas_confmat(confmat):
	TP = confmat[1, 1]
	TN = confmat[0, 0]
	FP = confmat[0, 1]
	FN = confmat[1, 0]
	accuracy = (TP + TN) / (TP + TN + FP + FN)
	precision = TP / (TP + FP)
	recall = TP / (TP + FN)
	specificity = TN / (TN + FP)
	nombres_columnas = ['TP','TN','FP','FN','accuracy','precision','recall','specificity']
	metricas = pd.DataFrame([[TP,TN,FP,FN,accuracy,precision,recall,specificity]], columns=nombres_columnas)
	return metricas


## Prueba de SVM con kernel rbf

In [11]:
#Máquina de vectores de soporte
modelo_svm = svm.SVC(kernel='rbf', C=1.0, gamma='scale')

In [12]:
modelo = modelo_svm
i = 0
X_train = list_X_train[i]
y_train = list_y_train[i]
X_test = list_X_test[i]
y_test = list_y_test[i]
modelo.fit(X_train,y_train)
pred_train = modelo.predict(X_train)
confmat_train = confusion_matrix(y_train, pred_train)
pred_test = modelo.predict(X_test)
confmat_test = confusion_matrix(y_test, pred_test)

In [13]:
metricas_train = obtener_metricas_confmat(confmat_train)
metricas_test = obtener_metricas_confmat(confmat_test)

In [14]:
print(f'Métricas de predicción con datos de entrenamiento:')
print(metricas_train)
print(f'\nMétricas de predicción con datos de prueba:')
print(metricas_test)

Métricas de predicción con datos de entrenamiento:
     TP     TN    FP     FN  accuracy  precision    recall  specificity
0  1814  15467  1555  10329  0.592525   0.538439  0.149386     0.908648

Métricas de predicción con datos de prueba:
    TP    TN   FP    FN  accuracy  precision    recall  specificity
0  456  3768  375  2693  0.579265   0.548736  0.144808     0.909486


### Los resultados no son alentadores, por lo tanto no se siguió con el K-fold validation

Métricas de predicción con datos de entrenamiento:
     TP     TN    FP     FN  accuracy  precision    recall  specificity
0  1814  15467  1555  10329  0.592525   0.538439  0.149386     0.908648

Métricas de predicción con datos de prueba:
    TP    TN   FP    FN  accuracy  precision    recall  specificity
0  456  3768  375  2693  0.579265   0.548736  0.144808     0.909486